In [11]:
import time
from llama_cpp import Llama
import re

In [2]:
llm = Llama(
  model_path="/Volumes/Rishik T7_1/AI/LLMs/Phi-3-mini-4k-instruct-q4.gguf",
  n_ctx=4096,  # The max sequence length to use
  n_threads=8,
  n_gpu_layers=-1
)

llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from /Volumes/Rishik T7_1/AI/LLMs/Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 32064
llama_model_loader: - kv   3:                       llama.context_length u32              = 4096
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 3072
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 8192
llama_model_loader: - kv   7:                 llam

llm_load_vocab: special tokens definition check successful ( 323/32064 ).
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = SPM
llm_load_print_meta: n_vocab          = 32064
llm_load_print_meta: n_merges         = 0
llm_load_print_meta: n_ctx_train      = 4096
llm_load_print_meta: n_embd           = 3072
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 32
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_rot            = 96
llm_load_print_meta: n_embd_head_k    = 96
llm_load_print_meta: n_embd_head_v    = 96
llm_load_print_meta: n_gqa            = 1
llm_load_print_meta: n_embd_k_gqa     = 3072
llm_load_print_meta: n_embd_v_gqa     = 3072
llm_load_print_meta: f_norm_eps       = 0.0e+00
llm_load_print_meta: f_norm_rms_eps   = 1.0e-05
llm_load_print_meta: f_clamp_kqv      = 0.0e+00
llm_load_print_meta: f_max_alibi_bias = 0.0e+00
llm_load_print_meta

In [107]:
# prompt = "What is 1+1=?"
# prompt = '''
#     Your job is to extract action and objects from the given task. Respond only in JSON format.
#     \nDo not hello, comment, explain, or elaborate your thought processes. Your response should have two keys: actions and objects. Split the task in as many as sub-actions possible. Only mention the name of object.
#     \n
#     \nExample 1:
#     \nTask: Pick up banana and place on the plate.
#     \nYou: {"actions": ["pick", "place"], objects: ["banana", "plate"]}
#     \n
#     \nExample 2:
#     \nTask: Pick up the blue box.
#     \nYou: {"actions": ["pick"], objects: ["blue box"]}
#     \n
#     \n Question: Pick up the flower pot and place it inside the box.
#     '''

# taskQuestion = 'throw the coke cane in the garbage bin'
# taskQuestion = 'Put the flower pot on the table'
# taskQuestion = 'Pick up the red ball and place it on the glass'
# taskQuestion = 'pickup the banana and dance'
taskQuestion = 'Hold the soldering iron'
# taskQuestion = 'Keep the apple in the tiffin box'
# taskQuestion = 'सेब को टिफिन बॉक्स में रखें'
# taskQuestion = 'Tartsa az almát a tiffin dobozban' # hungarian
# taskQuestion = 'Vedd fel a piros labdát és tedd a dobozba' # hungarian
# taskQuestion = 'Guarda la manzana en la caja tiffin.' # spanish
# taskQuestion = 'Recoge la pelota y colócala en la caja' # spanish
# taskQuestion = 'Conserva la mela nella scatola del tiffin' # italian

prompt = '''
    Your job is to extract actions and objects from the given task and invoke the allowed functions sequentially. Answer only with action.ignore() if no appropriate function available or you don't know what to do.
    Do not greet, comment, explain, or elaborate your thought processes. Split the task in as many as sub-actions possible. Use semicolon to seperate the function calls.
    Always recognise task language and translate the task and the object name to English.
    Allowed functions are: action.pick(), action.pick_place(), action.ignore()
    \nExample 1:
    Task: Pick up banana and place on the plate.
    Answer: action.pick_place("banana", "plate")
    \nExample 2:
    Task: Pickup the blue box.
    Answer: action.pick("blue box")
    \nNow translate to English and answer for Task: ''' + taskQuestion

start_time = time.perf_counter()
output = llm(
  f"<|user|>\n{prompt}<|end|>\n<|assistant|>",
  max_tokens=160,  # Generate up to 256 tokens
  stop=["<|end|>"],
  echo=False,
  temperature=0.3
)
end_time = time.perf_counter()
print("inference time: %.1fms\n" % ((end_time-start_time)*1000,))

print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =   10977.71 ms
llama_print_timings:      sample time =       1.12 ms /    10 runs   (    0.11 ms per token,  8944.54 tokens per second)
llama_print_timings: prompt eval time =     250.63 ms /     9 tokens (   27.85 ms per token,    35.91 tokens per second)
llama_print_timings:        eval time =     220.48 ms /     9 runs   (   24.50 ms per token,    40.82 tokens per second)
llama_print_timings:       total time =     489.31 ms /    18 tokens


inference time: 492.4ms

 action.pick("soldering iron");


In [14]:
cleanText = output['choices'][0]['text']
newLineIdx = cleanText.find('\n')
if newLineIdx != -1:
    cleanText = cleanText[:newLineIdx]

cleanText = re.sub('[^A-Za-z0-9\"\':\{\}\[\]\_\-\,\.\(\)\;\s]+', '', cleanText)
cleanText = cleanText.strip(' ')
print(cleanText)

# handle "target location" as args in place()

action.pick("red ball"); action.place("box")


In [19]:
actions = cleanText.split(';')
actions = [a.strip() for a in actions if a != '']
print(actions)

['action.pick("red ball")', 'action.place("box")']


#### Tried chat_completion, but that works poorly and has slow inference.